# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fa408379dc0>
├── 'a' --> tensor([[ 0.7380, -0.5792, -1.0891],
│                   [-0.3249,  0.0254, -1.3844]])
└── 'x' --> <FastTreeValue 0x7fa408379d30>
    └── 'c' --> tensor([[ 0.8517, -0.3993,  1.1132, -0.5886],
                        [-1.1397, -1.2686,  1.0152,  0.3250],
                        [-0.7803,  0.9864, -2.5061,  0.7014]])

In [4]:
t.a

tensor([[ 0.7380, -0.5792, -1.0891],
        [-0.3249,  0.0254, -1.3844]])

In [5]:
%timeit t.a

58.5 ns ± 0.0489 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fa408379dc0>
├── 'a' --> tensor([[ 1.6498,  1.2412, -0.4433],
│                   [ 1.2421, -0.7454, -0.3682]])
└── 'x' --> <FastTreeValue 0x7fa408379d30>
    └── 'c' --> tensor([[ 0.8517, -0.3993,  1.1132, -0.5886],
                        [-1.1397, -1.2686,  1.0152,  0.3250],
                        [-0.7803,  0.9864, -2.5061,  0.7014]])

In [7]:
%timeit t.a = new_value

57.8 ns ± 0.0283 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.7380, -0.5792, -1.0891],
               [-0.3249,  0.0254, -1.3844]]),
    x: Batch(
           c: tensor([[ 0.8517, -0.3993,  1.1132, -0.5886],
                      [-1.1397, -1.2686,  1.0152,  0.3250],
                      [-0.7803,  0.9864, -2.5061,  0.7014]]),
       ),
)

In [10]:
b.a

tensor([[ 0.7380, -0.5792, -1.0891],
        [-0.3249,  0.0254, -1.3844]])

In [11]:
%timeit b.a

61.3 ns ± 0.0247 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.8228,  0.3203, -0.1011],
               [ 1.6778, -1.4378, -1.1524]]),
    x: Batch(
           c: tensor([[ 0.8517, -0.3993,  1.1132, -0.5886],
                      [-1.1397, -1.2686,  1.0152,  0.3250],
                      [-0.7803,  0.9864, -2.5061,  0.7014]]),
       ),
)

In [13]:
%timeit b.a = new_value

492 ns ± 0.116 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

842 ns ± 0.0648 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 16.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

146 µs ± 1.17 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 444 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fa35d637f70>
├── 'a' --> tensor([[[ 0.7380, -0.5792, -1.0891],
│                    [-0.3249,  0.0254, -1.3844]],
│           
│                   [[ 0.7380, -0.5792, -1.0891],
│                    [-0.3249,  0.0254, -1.3844]],
│           
│                   [[ 0.7380, -0.5792, -1.0891],
│                    [-0.3249,  0.0254, -1.3844]],
│           
│                   [[ 0.7380, -0.5792, -1.0891],
│                    [-0.3249,  0.0254, -1.3844]],
│           
│                   [[ 0.7380, -0.5792, -1.0891],
│                    [-0.3249,  0.0254, -1.3844]],
│           
│                   [[ 0.7380, -0.5792, -1.0891],
│                    [-0.3249,  0.0254, -1.3844]],
│           
│                   [[ 0.7380, -0.5792, -1.0891],
│                    [-0.3249,  0.0254, -1.3844]],
│           
│                   [[ 0.7380, -0.5792, -1.0891],
│                    [-0.3249,  0.0254, -1.3844]]])
└── 'x' --> <FastTreeValue 0x7fa363c25ee0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.4 µs ± 64.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fa363e9a280>
├── 'a' --> tensor([[ 0.7380, -0.5792, -1.0891],
│                   [-0.3249,  0.0254, -1.3844],
│                   [ 0.7380, -0.5792, -1.0891],
│                   [-0.3249,  0.0254, -1.3844],
│                   [ 0.7380, -0.5792, -1.0891],
│                   [-0.3249,  0.0254, -1.3844],
│                   [ 0.7380, -0.5792, -1.0891],
│                   [-0.3249,  0.0254, -1.3844],
│                   [ 0.7380, -0.5792, -1.0891],
│                   [-0.3249,  0.0254, -1.3844],
│                   [ 0.7380, -0.5792, -1.0891],
│                   [-0.3249,  0.0254, -1.3844],
│                   [ 0.7380, -0.5792, -1.0891],
│                   [-0.3249,  0.0254, -1.3844],
│                   [ 0.7380, -0.5792, -1.0891],
│                   [-0.3249,  0.0254, -1.3844]])
└── 'x' --> <FastTreeValue 0x7fa363c25850>
    └── 'c' --> tensor([[ 0.8517, -0.3993,  1.1132, -0.5886],
                        [-1.1397, -1.2686,  1.0152,  0.3250],
                 

In [23]:
%timeit t_cat(trees)

29.3 µs ± 34.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.4 µs ± 90.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.8517, -0.3993,  1.1132, -0.5886],
                       [-1.1397, -1.2686,  1.0152,  0.3250],
                       [-0.7803,  0.9864, -2.5061,  0.7014]],
              
                      [[ 0.8517, -0.3993,  1.1132, -0.5886],
                       [-1.1397, -1.2686,  1.0152,  0.3250],
                       [-0.7803,  0.9864, -2.5061,  0.7014]],
              
                      [[ 0.8517, -0.3993,  1.1132, -0.5886],
                       [-1.1397, -1.2686,  1.0152,  0.3250],
                       [-0.7803,  0.9864, -2.5061,  0.7014]],
              
                      [[ 0.8517, -0.3993,  1.1132, -0.5886],
                       [-1.1397, -1.2686,  1.0152,  0.3250],
                       [-0.7803,  0.9864, -2.5061,  0.7014]],
              
                      [[ 0.8517, -0.3993,  1.1132, -0.5886],
                       [-1.1397, -1.2686,  1.0152,  0.3250],
                       [-0.7803,  0.9864, -2.5061,  0.7014]],

In [26]:
%timeit Batch.stack(batches)

78.6 µs ± 102 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.8517, -0.3993,  1.1132, -0.5886],
                      [-1.1397, -1.2686,  1.0152,  0.3250],
                      [-0.7803,  0.9864, -2.5061,  0.7014],
                      [ 0.8517, -0.3993,  1.1132, -0.5886],
                      [-1.1397, -1.2686,  1.0152,  0.3250],
                      [-0.7803,  0.9864, -2.5061,  0.7014],
                      [ 0.8517, -0.3993,  1.1132, -0.5886],
                      [-1.1397, -1.2686,  1.0152,  0.3250],
                      [-0.7803,  0.9864, -2.5061,  0.7014],
                      [ 0.8517, -0.3993,  1.1132, -0.5886],
                      [-1.1397, -1.2686,  1.0152,  0.3250],
                      [-0.7803,  0.9864, -2.5061,  0.7014],
                      [ 0.8517, -0.3993,  1.1132, -0.5886],
                      [-1.1397, -1.2686,  1.0152,  0.3250],
                      [-0.7803,  0.9864, -2.5061,  0.7014],
                      [ 0.8517, -0.3993,  1.1132, -0.5886],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 126 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

310 µs ± 3.71 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
